<a href="https://colab.research.google.com/github/darshandugar2004/CorporateMailHandler/blob/main/Bert_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corporate Email Categorization Model

This is a fine-tuned DistilBERT model for classifying corporate emails into various categories.

## Overview

This project focuses on building a robust model for corporate email categorization to automate the sorting and routing of internal and external communications. The process involves standard NLP techniques, data augmentation for handling class imbalance, and fine-tuning a pre-trained transformer model.

## Approach and Methodology

The fine-tuning process for this model followed these key steps:

1.  **Data Loading**: The initial dataset was loaded from the specified source.
2.  **Exploratory Data Analysis (EDA)**: Basic EDA was performed to understand the data structure, identify missing values, and analyze the distribution of labels.
3.  **Handling Data Imbalance**: It was observed that some categories had significantly fewer examples than others. To address this, data augmentation was employed.
4.  **Artificial Data Generation**: Synthetic email examples were generated for under-represented classes using the Groq API and the Llama3-8b-8192 model. This helped to balance the dataset and improve the model's ability to learn from minority classes.
5.  **Encoding**: String labels were converted into numerical representations (integer IDs) required for model training.
6.  **Tokenization**: The email text data was tokenized using the `distilbert-base-uncased` tokenizer. This process converts the text into a sequence of numerical tokens that the model can process, including adding special tokens and attention masks.
7.  **Data Splitting**: The combined original and augmented dataset was split into training, validation, and test sets to facilitate model training and evaluation.
8.  **Model Initialization**: A pre-trained `distilbert-base-uncased` model was loaded. For the classification task, output layers were added to the model to predict the defined categories.
9.  **Training**: The model was fine-tuned on the augmented training dataset using the Hugging Face `Trainer`. The training involved optimizing the model's parameters to minimize the classification loss.
10. **Evaluation**: The fine-tuned model's performance was evaluated on a separate, unseen test set.

## Training Details

*   **Base Model**: `distilbert-base-uncased`
*   **Training Method**: Fine-tuning with added output layers for sequence classification. (While LoRA was mentioned, the provided notebook code does not explicitly show LoRA implementation. This description reflects the standard fine-tuning with classification layers as seen in the code.)
*   **Epochs**: 5
*   **Batch Size**: 16 (Training and Evaluation)
*   **Optimizer**: AdamW
*   **Learning Rate Scheduler**: Linear warmup
*   **Loss Function**: Cross-Entropy Loss (implicitly used by `AutoModelForSequenceClassification`)

## Evaluation Results

The model's performance on the final test set is summarized below:

| Metric             | Value    |
|--------------------|----------|
|accuracy           |  0.9130 |
| f1_weighted        |  0.7949 |
| precision_weighted |  0.8462 |
| recall_weighted    |  0.7692




In [ ]:
# Check if GPU is available (highly recommended for faster training)
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. Training might be slow.")

# 1. Install necessary libraries
!pip install -qq transformers datasets accelerate scikit-learn pandas tabulate groq

# Imports
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tabulate import tabulate # Import tabulate for pretty printing

# Set a seed for reproducibility
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [ ]:
ds = load_dataset("infinite-dataset-hub/CorporateMailCategorization")

# Convert to pandas DataFrame for easier null handling
df = ds["train"].to_pandas()

In [ ]:
initial_rows = len(df)
df_cleaned = df.dropna(subset=['label']).copy()
rows_dropped = initial_rows - len(df_cleaned)
print(f"\nDropped {rows_dropped} rows with null 'label' values.")
print(f"Remaining rows after dropping nulls: {len(df_cleaned)}")
df_cleaned

In [ ]:
is_duplicate_counts = df_cleaned.duplicated().value_counts()
print(is_duplicate_counts)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset

label_counts = df_cleaned['label'].value_counts()
label_counts

In [ ]:
# Important: Map string labels to integers for the model
# Get unique labels from the cleaned dataset
unique_labels = sorted(df_cleaned['label'].unique().tolist())
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for i, label in enumerate(unique_labels)}

print(f"\nDetected labels and their mappings: {label_to_id}")
num_labels = len(unique_labels)
print(f"Number of unique labels: {num_labels}")

# Apply label mapping to the cleaned dataset
def map_labels_to_ids(example):
    example['label'] = label_to_id[example['label']]
    return example

# Convert cleaned DataFrame to Hugging Face Dataset
full_labeled_ds = Dataset.from_pandas(df_cleaned).map(map_labels_to_ids)

In [ ]:
id_to_label
label_to_id

In [ ]:
import random
import os
import json
from groq import Groq

from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

class SyntheticTextGenerator:
    def __init__(self, id_to_label_map):
        self.client = Groq()
        self.groq_model = "llama-3.1-8b-instant"
        self.id_to_label = id_to_label_map # Store the mapping for better prompts

    def generate_similar_texts(self, reference_texts: list, num_to_generate: int, target_label_id: int) -> list:
        """
        Generates new text examples similar to reference_texts for a given label,
        expecting JSON output from the LLM.
        """
        target_label_name = self.id_to_label.get(target_label_id, "Unknown Category")

        if not reference_texts:
            print(f"Warning: No reference texts provided for label ID {target_label_id} ({target_label_name}). Cannot generate examples.")
            return []

        # Prepare reference examples for the prompt
        reference_json_examples = []
        for text in reference_texts:
            # Ensure reference examples are also clean JSON lines
            reference_json_examples.append(json.dumps({"text": text, "label": target_label_name}))
        reference_str = ",\n".join(reference_json_examples)

        # Adjusted system prompt for more robust JSON output instructions
        system_prompt = (
            "You are an AI assistant specialized in generating realistic and diverse corporate email snippets "
            "for data augmentation. Your output MUST be a strict JSON array where each element is a JSON object with 'text' and 'label' keys. "
            "It must start with `[` and end with `]`. "
            "Do NOT include any additional text, explanations, or formatting outside of the JSON array. "
            "Do NOT wrap the array in any other JSON object, like {\"data\": [...]}. Provide ONLY the JSON array. "
            "Ensure the generated emails are plausible and distinct from the references."
            "Always wrap your JSON output in triple backticks with 'json' language specifier, e.g., ```json [...]```."
        )

        user_prompt = f"""
        Generate exactly {num_to_generate} new and distinct corporate email snippets.
        Each snippet must be categorized as '{target_label_name}'.

        Here are some existing examples for reference, categorized as '{target_label_name}'.
        These are provided as a JSON array of objects, with 'text' and 'label' keys:
        ```json
        [
        {reference_str}
        ]
        ```

        Produce your output as a single JSON array, directly.
        Each element in the array must be a JSON object with two keys:
        1. "text": The generated email snippet.
        2. "label": The category, which must be '{target_label_name}'.

        Output ONLY the JSON array, and wrap it in triple backticks with the 'json' language specifier (e.g., ```json [...]``` ).
        For example:
        ```json
        [
          {{"text": "Generated email snippet 1", "label": "{target_label_name}"}},
          {{"text": "Generated email snippet 2", "label": "{target_label_name}"}}
        ]
        ```
        """

        json_output_str = None # Initialize json_output_str

        try:
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                model=self.groq_model,
                temperature=0.8,
                max_tokens=int(num_to_generate * 100)
            )

            # Extract content and try to clean it
            json_output_str = chat_completion.choices[0].message.content.strip()
            # print(json_output_str)

            if json_output_str.startswith("```") :
                cleaned_json_str = json_output_str[len("```"):].strip()
                print('check 0')
            if cleaned_json_str.startswith("json") :
                cleaned_json_str = cleaned_json_str[len("json"):].strip()
                print('check 1')
            if cleaned_json_str.endswith("```") :
                cleaned_json_str = cleaned_json_str[:-len("```")].strip()
                print('check 2')
            else:
                # If it didn't use backticks, try parsing directly as the model might still try to output raw JSON
                # cleaned_json_str = json_output_str
                print('check 3')
            # print('out', cleaned_json_str)

            parsed_raw = None
            try:
                parsed_raw = json.loads(cleaned_json_str)
            except json.JSONDecodeError as de:
                print(f"Secondary JSON parsing failed: {de}")
                raise json.JSONDecodeError("Failed to parse JSON after initial cleaning attempts.", cleaned_json_str, 0)


            # After parsing (which might have been a dict or a list)
            generated_data_list = []
            if isinstance(parsed_raw, dict) and 'data' in parsed_raw and isinstance(parsed_raw['data'], list):
                generated_data_list = parsed_raw['data']
                print("Note: LLM output was wrapped in 'data' key, extracted successfully.")
            elif isinstance(parsed_raw, list):
                generated_data_list = parsed_raw
            else:
                raise ValueError(f"LLM output is not a direct JSON array or wrapped in a 'data' key as expected. Type: {type(parsed_raw)}")

            # Validate the structure of each item
            validated_examples = []
            for item in generated_data_list:
                if isinstance(item, dict) and 'text' in item and 'label' in item:
                    if item['label'] == target_label_name:
                        validated_examples.append(item)
                    else:
                        print(f"Warning: Generated label '{item['label']}' does not match target '{target_label_name}'. Skipping.")
                else:
                    print(f"Warning: Invalid item structure received from LLM: {item}. Skipping.")
            return validated_examples

        except json.JSONDecodeError as e:
            print(f"Error parsing JSON from Groq API for {target_label_name}: {e}")
            print(f"Raw LLM output (for debugging): {json_output_str}")
            return []
        except Exception as e:
            print(f"Error calling Groq API or processing response for {target_label_name}: {e}")
            print(f"Raw LLM output (if available): {json_output_str}")
            return []


# Instantiate the synthetic data generator
generator = SyntheticTextGenerator(id_to_label)

label_counts = df_cleaned['label'].value_counts()
under_represented_label_names = label_counts[label_counts < 15].index.tolist()

augmented_data = []
temp = ['Financial Report']

for label_name in under_represented_label_names:
    print(f"\nAugmenting label: '{label_name}'")
    current_count = label_counts[label_name]

    # Get existing examples for reference
    reference_df = df_cleaned[df_cleaned['label'] == label_name]

    num_references = min(5, len(reference_df)) # Use up to 5 references
    reference_texts = random.sample(reference_df['text'].tolist(), num_references)

    # Determine how many new examples are needed to reach TARGET
    num_to_generate = 15 - current_count

    print(f"  Need to generate {num_to_generate} new examples for '{label_name}'.")

    label_id = label_to_id[label_name]
    new_generated_examples = generator.generate_similar_texts(reference_texts, num_to_generate, label_id)

    augmented_data.extend(new_generated_examples)
    print(f"  Generated {len(new_generated_examples)} new examples for '{label_name}'.")

# Now, 'augmented_data' contains a list of dictionaries, each with 'text' and 'label'
print(f"\n--- Overall Generated Synthetic Data ---")
print(f"Total new synthetic examples generated across all under-represented labels: {len(augmented_data)}")
if augmented_data:
    # Print a sample of the generated data to verify format
    print("\nSample of generated data (first 3):")
    for i, ex in enumerate(augmented_data[:3]):
        print(f"  {i+1}. Text: '{ex['text'][:70]}...', Label: '{ex['label']}'")

In [ ]:
if augmented_data:
    augmented_df = pd.DataFrame(augmented_data)
    df_augmented = pd.concat([df_cleaned, augmented_df], ignore_index=True)
    print(f"Total rows after augmentation: {len(df_augmented)}")
    df_cleaned = df_augmented.copy()
else:
    print("\nNo data augmentation performed as no under-represented labels were found or generation failed.")

In [ ]:
def map_labels_to_ids_for_augmented(row):
    # Check if the 'label' in the row is a string before attempting to map
    if isinstance(row['label'], str):
        row['label'] = label_to_id[row['label']]
    return row

# Convert df_cleaned (now potentially df_augmented) to a Hugging Face Dataset
# Use .apply(..., axis=1) to apply the function row-wise
df_cleaned = df_cleaned.apply(map_labels_to_ids_for_augmented, axis=1)
# df_cleaned.head(100)

In [ ]:
# Now, convert the DataFrame to a Hugging Face Dataset
print("Final Dataset object created successfully:")
print(full_labeled_ds)

In [ ]:
full_labeled_ds=full_labeled_ds.remove_columns(["idx"])

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Move model to GPU if available
model.to(device)


In [ ]:
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    labels = np.array(p.label_ids).flatten()

    accuracy = accuracy_score(labels, predictions)
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
    precision_macro = precision_score(labels, predictions, average='macro', zero_division=0)
    recall_macro = recall_score(labels, predictions, average='macro', zero_division=0)


    metrics = {
        'accuracy': accuracy,
        'f1_weighted': f1_macro,
        'precision_weighted': precision_macro,
        'recall_weighted': recall_macro,
    }
    return metrics


In [ ]:
train_val_test_split = full_labeled_ds.train_test_split(test_size=0.1, seed=42)
train_val_ds = train_val_test_split["train"]
test_ds_final = train_val_test_split["test"] # This is our final, labeled test set

train_val_split = train_val_ds.train_test_split(test_size=0.11, seed=42)
train_ds_split = train_val_split["train"]
eval_ds_split = train_val_split["test"] # This is our validation set

print(f"\nDataset Splits:")
print(f"  Training samples: {len(train_ds_split)}")
print(f"  Validation samples: {len(eval_ds_split)}")
print(f"  Final Test samples: {len(test_ds_final)}")

print("\nTrain Dataset Split Structure:", train_ds_split)
print("Validation Dataset Split Structure:", eval_ds_split)
print("Final Test Dataset Structure:", test_ds_final)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train_ds = train_ds_split.map(tokenize_function, batched=True)
tokenized_eval_ds = eval_ds_split.map(tokenize_function, batched=True)
tokenized_test_ds_final = test_ds_final.map(tokenize_function, batched=True) # Tokenize final test set

# Remove original text and idx columns as they are no longer needed for training/evaluation
tokenized_train_ds = tokenized_train_ds.remove_columns(["text"])
tokenized_eval_ds = tokenized_eval_ds.remove_columns(["text"])
tokenized_test_ds_final = tokenized_test_ds_final.remove_columns(["text"]) # Keep label for final test

print(tokenized_train_ds)
# print(len(tokenized_eval_ds))
# print(len(tokenized_test_ds_final))

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_mail_category", # Directory for logs and checkpoints
    num_train_epochs=10,                  # Number of training epochs
    per_device_train_batch_size=16,      # Batch size for training
    per_device_eval_batch_size=16,       # Batch size for evaluation
    warmup_steps=10,                     # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                   # Strength of weight decay
    logging_dir="./logs_mail_category",  # Directory for storing logs
    logging_strategy="epoch",            # Log metrics at the end of each epoch
    save_strategy="epoch",               # Save model at the end of each epoch
    eval_strategy="epoch",               # Evaluate at the end of each epoch
    load_best_model_at_end=True,         # Load the best model at the end of training
    metric_for_best_model="f1_weighted", # Metric to use to compare models
    report_to="none",                    # Don't report to any online services
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_eval_ds,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

# Start training!
trainer.train()

print("\n--- 7. Evaluation Metrics on Validation Set ---")
eval_results = trainer.evaluate()
print("\nEvaluation Metrics on Validation Set (used during training):")
eval_df = pd.DataFrame([eval_results]).transpose()
eval_df.columns = ['Value']
print(tabulate(eval_df, headers='keys', tablefmt='grid', floatfmt=".4f"))

In [ ]:
print("\n--- Saving the Fine-tuned Model ---")
save_path = "./mail_category"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print(f"\nModel and tokenizer saved to: {save_path}")